In [1]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
model_name = "inokufu/flaubert-base-uncased-xnli-sts-finetuned-education"

tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/598k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/305 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/582 [00:00<?, ?B/s]

In [3]:
lr = 2e-5
epochs =  6
batch_size = 5
max_seq_len = 75

test_frac = 0.1

In [4]:
import os

def set_seed(seed=106052):
    """Set seed for reproducibility.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [5]:
class CEFRDataset(Dataset):
    """Classification dataset, built on top of pytorch dataset object
    """
    
    def __init__(self, texts, labels):
        
        self.encoder = LabelEncoder()
        print(self.encoder.__dict__)
        self.texts = texts
        self.labels = self.encoder.fit_transform(labels)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoded_text = tokenizer(
            text,
            padding="max_length",
            max_length=max_seq_len,
            truncation=True,
            return_tensors="pt",
        )
        encoded_text["input_ids"] = encoded_text["input_ids"].squeeze()
        encoded_text["attention_mask"] = encoded_text["attention_mask"].squeeze()
        label = torch.tensor(label)

        return {
            "input_ids": encoded_text["input_ids"],
            "attention_mask": encoded_text["attention_mask"],
            "labels": label,
        }

    def get_labels(self):
        return self.labels

In [6]:
def train(train_set, valid_set, epochs=10, warmup_size=0.1, lr=1e-3, batch_size=16):
    model = get_model(model_name)
    optim = AdamW(model.parameters(), lr=lr)
    scheduler = get_scheduler(
        optim, warmup_size, round(len(train_set) / batch_size * epochs)
    )
    training_args = get_training_args(epochs, batch_size)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=valid_set,
        optimizers=[optim, scheduler],
        compute_metrics=compute_accuracy,
    )
    trainer.train()
    trainer.save_model()
    return trainer

In [7]:
def get_model(pretrained_checkpoint):
    model = AutoModelForSequenceClassification.from_pretrained(
        pretrained_checkpoint, num_labels=6
    )
    return model.to(device)

In [8]:
os.environ["WANDB_DISABLED"] = "true"


def get_scheduler(optimizer, warmup_size, total_steps):
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=round(total_steps * warmup_size),
        num_training_steps=total_steps,
    )
    return scheduler


def get_training_args(epochs, batch_size):
    return TrainingArguments(
        output_dir="./b",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        logging_steps=50,
        fp16=False,
        evaluation_strategy="epoch",
        eval_accumulation_steps=1,
        report_to=None,
        save_total_limit=1,
        load_best_model_at_end=True
    )


def compute_accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}



In [9]:
lr = 2e-5
epochs =  7
batch_size = 8
max_seq_len = 512

In [10]:
def split_valid(df, frac=0.01):
    
    val = pd.DataFrame()
    val["text"] = ""
    val["label"] = -1
    
    for i in df.label.unique():
        val = pd.concat([val, df[df.label == i].sample(frac=frac)])
        
    return df[~df.index.isin(val.index)].reset_index(drop=True) , val.reset_index(drop=True)

In [11]:
data=pd.read_csv("../input/fr-cerfr/dataset_to_be_annotated_-_dataset_to_be_annotated.csv")

In [12]:
train_set_df = pd.read_csv("../input/fr-cerfr/dataset_to_be_annotated_-_dataset_to_be_annotated.csv")
train_set_df.drop("Unnamed: 3", axis=1, inplace=True)

In [13]:
train_set_df = train_set_df[pd.notnull(train_set_df['cefr'])]
train_set_df = train_set_df.reset_index(drop=True)

In [14]:
train_set_df.columns=["text","lens","label"]
train_set_df = train_set_df[train_set_df.label != "-"]
train_set_df = train_set_df[["text", "label"]]

In [15]:
A1_A2=pd.read_csv("../input/a1-a2-ex/A1_A2_tran.csv")

In [16]:
train_set_df=pd.concat([train_set_df, A1_A2], ignore_index=True)

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()




train_set_df.text = train_set_df.text.apply(lambda x: x.replace("\r", "").replace("\n", " "))

# extra_df = pd.read_csv("../input/frenchcefr/french_mike_june.csv")
# extra_df.columns = ["text", "label", "label_"]
# extra_df = extra_df[["text", "label"]]
# extra_df.text = extra_df.text.astype(str)
#train_set_df = pd.concat([train_set_df, extra_df]).reset_index(drop=True)

train_set_df, valid_set_df = split_valid(train_set_df)

In [18]:
for i in range(train_set_df.shape[0]):
    if train_set_df.loc[i,"label"]=="c1":
        train_set_df.loc[i,"label"]="C1"

In [19]:
train_set_df.label.unique()

array(['C1', 'B2', 'B1', 'A2', 'C2', 'A1'], dtype=object)

In [20]:
train_set_df.label.value_counts()

A1    285
A2    273
B2     91
B1     50
C1     37
C2     12
Name: label, dtype: int64

In [21]:
train_set_df.label = le.fit_transform(train_set_df.label)
valid_set_df.label = le.transform(valid_set_df.label)

In [22]:
train_set_df.label.unique()

array([4, 3, 2, 1, 5, 0])

In [23]:
valid_set_df.label.nunique()

3

In [24]:
# train_set_df = train_set_df.sample(frac=1)

In [25]:
from tqdm import tqdm 

def predict(model, text):
    
    preds = []
    
    for i in tqdm(range(len(text))):
        tokenized = tokenizer(text[i:i+1], return_tensors="pt", truncation=True, max_length=512).to("cuda")
        pred = model(**tokenized)
        preds.append(pred.logits.argmax(-1).item())

    return preds

In [26]:
train_set = CEFRDataset(train_set_df["text"], train_set_df["label"])
valid_set = CEFRDataset(valid_set_df["text"], valid_set_df["label"])


trainer_second = train(train_set, valid_set, epochs=epochs, warmup_size=0.2, lr=lr, batch_size=batch_size)
model = trainer_second.model

{}
{}


Downloading:   0%|          | 0.00/549M [00:00<?, ?B/s]

Some weights of FlaubertForSequenceClassification were not initialized from the model checkpoint at inokufu/flaubert-base-uncased-xnli-sts-finetuned-education and are newly initialized: ['position_ids', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,1.396800,0.585634,0.714286,0.209700,33.384000
2,0.729700,0.626671,0.714286,0.201300,34.776000
3,0.508500,0.304570,0.714286,0.206500,33.893000
4,0.509200,0.366653,0.857143,0.322900,21.681000
5,0.229300,0.519231,0.857143,0.222500,31.463000
6,0.108700,0.500880,0.857143,0.203400,34.415000
7,0.055000,0.481383,0.857143,0.204400,34.255000


In [27]:
valid_set_df["preds"] = train_set.encoder.inverse_transform(predict(model, valid_set_df.text.tolist()))
valid_set_df.columns = ["text", "cefr", "preds",] 

100%|██████████| 7/7 [00:00<00:00, 36.66it/s]


In [28]:
lingua = pd.read_csv("../input/lingua/fr_lingua.csv")
lingua["preds"] = train_set.encoder.inverse_transform(predict(model, lingua.text.tolist()))

100%|██████████| 111/111 [00:02<00:00, 53.53it/s]


In [29]:
lingua.shape

(111, 3)

In [30]:
lingua.cefr.value_counts()

B1    60
A2    26
A1    13
B2    12
Name: cefr, dtype: int64

In [31]:
lingua.preds.unique()

array([2, 0, 3, 1, 4])

In [32]:
def compute_average_distance(df, col_name="cefr") :
    
    labels = ["A1", "A2", "B1", "B2", "C1", "C2"]
    return (df[col_name].apply(lambda x: labels.index(x)) - df.preds.apply(lambda x: labels.index(x))).abs().mean()


In [33]:
lingua.preds.value_counts()

2    70
3    22
0     9
4     7
1     3
Name: preds, dtype: int64

In [34]:
lingua["preds"] = le.inverse_transform(lingua.preds)

In [35]:
lingua.cefr.value_counts()

B1    60
A2    26
A1    13
B2    12
Name: cefr, dtype: int64

In [36]:
lingua.preds.value_counts()

B1    70
B2    22
A1     9
C1     7
A2     3
Name: preds, dtype: int64

In [37]:
print("Distance: ")
print(compute_average_distance(lingua, "cefr"))

print(f"Lingua accuracy: {(lingua['preds'] == lingua['cefr']).mean()}")

Distance: 
0.7837837837837838
Lingua accuracy: 0.40540540540540543


In [38]:
# for i in range(lingua.shape[0]):
#     if (lingua.loc[i,"preds"]=="C1") | (lingua.loc[i,"preds"]=="C2") :
#         lingua.loc[i,"Predss"]="B2"
#     else :
#         lingua.loc[i,"Predss"]= lingua.loc[i,"preds"]
# lingua["Predss"].value_counts()

In [39]:
# print("Distance: ")
# print(compute_average_distance(lingua, "cefr"))

# print(f"Lingua accuracy: {(lingua['Predss'] == lingua['cefr']).mean()}")

In [40]:
lingua.columns

Index(['text', 'cefr', 'preds'], dtype='object')

In [41]:
# lingua.columns=["text","cefr","preds:A1_A2_from_trans_data&new_data,acc:0.5"]
# lingua.to_csv("lingua1.csv", index=False)

****

Distance: 
0.990990990990991
Lingua accuracy: 0.40540540540540543 <br>
2    82
3    15
5    10
4     2
0     1
1     1 <br> 
inokufu/flaubert-base-uncased-xnli-sts-finetuned-education

****


Distance: 
0.8648648648648649
Lingua accuracy: 0.450450450450450462 <br>
3     7
1     5
5     4
4     4
0     1 <br> inokufu/flaubert-base-uncased-xnli-sts-finetuned-education